# Setup Opensearch for Multicloud E2E RAG demo

## Create index

In [ ]:
!pip install opensearch-py -q

In [ ]:
from opensearchpy import OpenSearch

import os

In [ ]:
host = os.environ['OPENSEARCH_HOST']
port = os.environ['OPENSEARCH_PORT']
auth = (
    os.environ['OPENSEARCH_USER'],
    os.environ['OPENSEARCH_PASSWORD']
) 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, 
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [ ]:
info = client.info()
print(f"Welcome to {info['version']['distribution']} {info['version']['number']}!")

In [ ]:
def delete_opensearch_index(opensearch_client, index_name):
    print(f"Trying to delete index {index_name}")
    try:
        response = opensearch_client.indices.delete(index=index_name)
        print(f"Index {index_name} deleted")
        return response['acknowledged']
    except Exception as e:
        print(f"Index {index_name} not found, nothing to delete")
        return True

def create_index(opensearch_client, index_name):
    settings = {
        "settings": {
            "index": {
                "knn": True
                }
            }
        }
    response = opensearch_client.indices.create(index=index_name, body=settings)
    return bool(response['acknowledged'])

def create_index_mapping(opensearch_client, index_name):
    response = opensearch_client.indices.put_mapping(
        index=index_name,
        body={
            "properties": {
                "vector_field": {
                    "type": "knn_vector",
                    "dimension": 384
                },
                "text": {
                    "type": "keyword"
                }
            }
        }
    )
    return bool(response['acknowledged'])

In [ ]:
INDEX_NAME = "rag_index"
FORCE_RECREATE = True

if FORCE_RECREATE:
    delete_opensearch_index(client, INDEX_NAME)

index_exists = client.indices.exists(index=INDEX_NAME)

if not index_exists:
    print("Creating OpenSearch index")
    index_created = create_index(client, INDEX_NAME)
    if index_created:
        print("Creating OpenSearch index mapping")
        success = create_index_mapping(client, INDEX_NAME)
        print(f"OpenSearch Index mapping created")
else:
    print("Opensearch index already exists")